# Input module examples

This is not yet perfect, but is getting there. The dataframes which are returned are designed to reflect MAGICC's underlying data input format. When we have pint with units (waiting on https://github.com/hgrecco/pint/pull/684) we can use `df.quantify()` and `df.dequantify()` to quickly add/remove units to our dataframes and swiftly do conversions etc. 

In [13]:
from os.path import join
from pprint import pprint

import pandas as pd
import pymagicc
from pymagicc.input import MAGICCInput

import expectexception

In [2]:
MAGICC6_DIR = join("..", "pymagicc", "MAGICC6", "run")

## Read files

In [3]:
mdata = MAGICCInput()
mdata.read(MAGICC6_DIR, "HISTRCP_CO2I_EMIS.IN")
mdata.head()

VARIABLE CO2I_EMIS                         
TODO           SET                         
UNITS          GtC                         
REGION      R5OECD R5REF R5ASIA R5MAF R5LAM
YEAR                                       
1765         0.003   0.0    0.0   0.0   0.0
1766         0.003   0.0    0.0   0.0   0.0
1767         0.003   0.0    0.0   0.0   0.0
1768         0.003   0.0    0.0   0.0   0.0
1769         0.003   0.0    0.0   0.0   0.0

In [4]:
pprint(mdata.metadata)

{'contact': 'Base year emissions inventories: Steve Smith (ssmith@pnl.gov) and '
            'Jean-Francois Lamarque (Jean-Francois.Lamarque@noaa.gov); RCP '
            '3-PD (IMAGE): Detlef van Vuuren (detlef.vanvuuren@pbl.nl); RCP '
            '4.5 (MiniCAM): Allison Thomson (Allison.Thomson@pnl.gov); RCP 6.0 '
            '(AIM): Toshihiko Masui (masui@nies.go.jp); RCP 8.5 (MESSAGE): '
            'Keywan Riahi (riahi@iiasa.ac.at); Concentrations & Forcing '
            'compilation: Malte Meinshausen (malte.meinshausen@pik-potsdam.de)',
 'data': 'Historical fossil&industrial CO2 (CO2I) Emissions '
         '(HISTRCP_CO2I_EMIS)',
 'header': 'RCPTOOL - MAGICC 6.X DATA FILE \n'
           'VERSION:   ALPHA - FIRST DRAFT - PRIMAP xls file written on 02 Sep '
           '2009, 17:04:37 \n'
           ' \n'
           ' \n'
           'DATA:  Historical fossil&industrial CO2 (CO2I) Emissions '
           '(HISTRCP_CO2I_EMIS) \n'
           'SOURCE:  RCP data as presented on '
         

## Accessors

`MAGICCInput` is built to make data accessing as simple as possible. As we have unique keys for each of our headers, we allow users to select data simply using lists, without specifying what each value should correspond to etc. (essentially we hide away all the pandas fiddly stuff).

Of course the underlying dataframe is always accessible via the `.df` attribute.

In [5]:
mdata['CO2I_EMIS'].head(2)

VARIABLE CO2I_EMIS                         
TODO           SET                         
UNITS          GtC                         
REGION      R5OECD R5REF R5ASIA R5MAF R5LAM
YEAR                                       
1765         0.003   0.0    0.0   0.0   0.0
1766         0.003   0.0    0.0   0.0   0.0

In [6]:
mdata['R5ASIA', 'CO2I_EMIS'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1765,0.0
1766,0.0


In [7]:
# changing order doesn't matter
mdata['CO2I_EMIS', 'R5ASIA'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1765,0.0
1766,0.0


In [8]:
%%expect_exception KeyError
# the above doesn't work on the raw dataframe
# because you need to specify all the levels 
# in the right order
mdata.df['R5ASIA', 'CO2I_EMIS']

---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-8-b7d9f3057b51> in <module>()
      2 # because you need to specify all the levels
      3 # in the right order
----> 4 mdata.df['R5ASIA', 'CO2I_EMIS']

~/Documents/AGCEC/MCastle/pymagicc/venv/lib/python3.6/site-packages/pandas/core/frame.py in __getitem__(self, key)
   2684             return self._getitem_frame(key)
   2685         elif is_mi_columns:
-> 2686             return self._getitem_multilevel(key)
   2687         else:
   2688             return self._getitem_column(key)

~/Documents/AGCEC/MCastle/pymagicc/venv/lib/python3.6/site-packages/pandas/core/frame.py in _getitem_multilevel(self, key)
   2728 
   2729     def _getitem_multilevel(self, key):
-> 2730         loc = self.columns.get_loc(key)
   2731         if isinstance(loc, (slice, Series, np.ndarray, Index)):
   2732             new_columns = self.colum

In [9]:
# can also put index values in
mdata[1956, 'CO2I_EMIS', 'R5ASIA'].head(2)

VARIABLE,CO2I_EMIS
TODO,SET
UNITS,GtC
REGION,R5ASIA
YEAR,
1956,0.110425


In [10]:
%%expect_exception KeyError
# although slicing fails, one for the to do list...
mdata[1956:1970, 'CO2I_EMIS', 'R5ASIA'].head(2)

TypeError: (slice(1956, 1970, None), 'CO2I_EMIS', 'R5ASIA')

In [11]:
mdata[1998]

VARIABLE CO2I_EMIS                                        
TODO           SET                                        
UNITS          GtC                                        
REGION      R5OECD     R5REF    R5ASIA     R5MAF     R5LAM
YEAR                                                      
1998      3.184738  0.853781  1.648214  0.522509  0.378757

## Writing files

Once you have your data in the format as above, writing files is trivial. 

In [12]:
mdata.write("HISTEXAMPLE_CO2I_EMIS.IN")

AssertionError: 

However, note that the format to write in is determined by the filename. Hence you can't just any old filename, it has to follow MAGICC's (somewhat cryptic) internal conventions.

In [ ]:
%%expect_exception ValueError
mdata.write("histexample.txt")